In [ ]:
# These imports let you use opencv
import cv2 #opencv itself
import numpy as np # matrix manipulations

#the following are to do with this interactive notebook code
%matplotlib inline
from matplotlib import pyplot as plt # this lets you draw inline pictures in the notebooks
import pylab # this allows you to control figure size
pylab.rcParams['figure.figsize'] = (5, 5) # this controls figure size in the notebook

In [ ]:
image=cv2.imread('data/bacillus/22727.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.show()

In [ ]:
import json
annot_file_name = "data/bacillus/22727.json"
with open(annot_file_name, 'r') as json_file:
    data = json.load(json_file)
data

In [ ]:
data['userAddedAnnotations']

In [ ]:
to_show = image.copy()
for r in data['userAddedAnnotations']:
    t, l, w, h = int(r['top']), int(r['left']), int(r['width']), int(r['height']) 
    cv2.rectangle(to_show, (l, t), (l+w, t+h), (255, 255, 0), 2)
    
plt.imshow(to_show)
plt.show()

In [ ]:
# Zoom a region
region = to_show[:300, :300]
region = cv2.resize(region, (600, 600))
plt.imshow(region)
plt.show()

## Detect bacillus

In [ ]:
transformed = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
h, s, v=cv2.split(transformed)
plt.imshow(image)
plt.show()
plt.imshow(h)

In [ ]:
is_bacillus = cv2.inRange(h, 130, 160)
plt.imshow(is_bacillus)

In [ ]:
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(is_bacillus, connectivity=8)

# Define los valores mínimo y máximo para el número de elementos en una componente conectada
min_area = 30  # Valor mínimo
max_area = 300  # Valor máximo

# Itera sobre las etiquetas
for label in range(1, num_labels):
    area = stats[label, cv2.CC_STAT_AREA]
    if min_area <= area <= max_area:
        # La componente conectada está dentro del rango deseado
        continue
    else:
        # Elimina la componente conectada configurando todos los píxeles en 0
        is_bacillus[labels == label] = 0

plt.imshow(image)
plt.show()
plt.imshow(is_bacillus)

In [ ]:
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(is_bacillus, connectivity=8)

# Define los valores mínimo y máximo para el número de elementos en una componente conectada
min_area = 30  # Valor mínimo
max_area = 300  # Valor máximo

boxes = []

# Itera sobre las etiquetas
for label in range(1, num_labels):
    area = stats[label, cv2.CC_STAT_AREA]
    if min_area <= area <= max_area:
        left = stats[label, cv2.CC_STAT_LEFT]
        top = stats[label, cv2.CC_STAT_TOP]
        width = stats[label, cv2.CC_STAT_WIDTH]
        height = stats[label, cv2.CC_STAT_HEIGHT]
        boxes.append((left, top, width, height))
        # La componente conectada está dentro del rango deseado
        continue
    else:
        # Elimina la componente conectada configurando todos los píxeles en 0
        is_bacillus[labels == label] = 0

delta = 3
to_show = image.copy()
for l, t, w, h in boxes:
    cv2.rectangle(to_show, (l-delta, t-delta, w+2*delta, h+2*delta), (255, 255, 0), 1)
plt.imshow(to_show)


Is there a parameter combination that performs better with most images?

You need first to label many images, so you can automatically evaluate configurations, and use gradient descent on the parameters??